In [2]:
import pandas as pd
import os
import openpyxl
import numpy as np
import re
from datetime import datetime
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
import tabulate
lemmatizer = WordNetLemmatizer()

In [777]:
class FinancialReport():
    def __init__(self,file):
        self.file = file
        self.readFile()
        self.findBalanceSheet()
        self.findCashFlowSheet()
        self.findIncomeSheet()
    def readFile(self):
        path = os.path.join("financial statements",self.file)
        self.statements = pd.read_excel(path,sheet_name=None)
    def getSheetNames(self):
        return self.statements.keys()
    
    def findBalanceSheet(self):
        balanceSheetName = self.getSheetWithTitle(["BALANCE"])[0]
        self.balance = self.statements[balanceSheetName]
        self.balance.columns = ["label"]+list(self.balance.columns[1:])
        
    def findIncomeSheet(self):
        incomeSheetName = self.getSheetWithTitle(["OPERATIONS","INCOME"])[0]
        self.income = self.statements[incomeSheetName]
        self.income = self.setFirstRowAsHeader(self.income)
        self.income.columns = ["label"]+list(self.income.columns[1:])
        self.income = self.income.loc[:, self.income.columns.notna()]
        if len(self.income.columns) >= 5:
            self.income = self.income.iloc[:,:-2]

        
    def getSheetWithTitle(self,words): 
        return [name for name in self.getSheetNames() if np.any([word in str(self.statements[name].columns[0]) for word in words]) ]
                                                                
    def setFirstRowAsHeader(self, sheet):
        sheet.columns = sheet.iloc[0]
        sheet = sheet.drop(0)
        return sheet
    
    def findCashFlowSheet(self):
        cashflowSheetName = self.getSheetWithTitle(["CASH"])[0]
        self.cashflow = self.statements[cashflowSheetName]
        self.cashflow = self.setFirstRowAsHeader(self.cashflow)
        self.cashflow.columns = ["label"]+list(self.cashflow.columns[1:])
        self.cashflow = self.cashflow.loc[self.cashflow.label!="Document Period End Date"]
    def getBalanceSheet(self):
        return self.balance
    def getCashFlowSheet(self):
        return self.cashflow
    def getIncomeSheet(self):
        return self.income
    

In [778]:
FinancialReport("10k 2018 (2017).xlsx").income

,label,"Feb. 03, 2018","Jan. 28, 2017","Jan. 30, 2016"
1,Net sales,15855,15516,15797
2,Cost of goods sold and occupancy expenses,9789,9876,10077
3,Gross profit,6066,5640,5720
4,Operating expenses,4587,4449,4196
5,Operating income,1479,1191,1524
6,Interest expense,74,75,59
7,Interest income,-19,-8,-6
8,Income before income taxes,1424,1124,1471
9,Income taxes,576,448,551
10,Net income,848,676,920


In [779]:
class HelperFunctions:
    def clean_up_string(self,string):
        string = string.strip()
        space_positions = [x.start() for x in re.finditer('\ ',  string)]
        for i in reversed(space_positions):
            if string[i+1:] in ["of", "and"]:
                string = string[:i]
        return string
    
    def merge_number(self,a,b):
        if pd.isnull(a):
            return b
        if pd.isnull(b):
            return a
        return max(a,b)
    
    def getFiles(self):
        files = os.listdir("financial statements")
        files.remove(".DS_Store")
        files = [file for file in files if file[0] != "~"]
        return files

    def getFinancialStatements(self):
        files = self.getFiles()
        FinancialReports = [FinancialReport(file) for file in files]
        return FinancialReports
        

In [780]:
class ProcessQuarters:
    
    def getQuarterHeaders(self):
        quarters = [col for col in list(self.finalStatement.columns) if col not in ["title","label","title_order"]]
        return quarters
    def includesYear(self, string):
        return string[-4:].isdigit()
        
    def getLabelHeaders(self):
        headers = [col for col in list(self.finalStatement.columns) if not self.includesYear(col)]
        return headers
    
    def setQuarters(self):
        dates = self.getQuarterHeaders()
        headers = self.getLabelHeaders()
        dates.sort(key=lambda date: datetime.strptime(date.replace(".",""), '%b %d, %Y'))
        self.finalStatement = self.finalStatement[headers+dates]
        self.finalStatement.columns = headers+[self.parseQuarter(date) for date in self.getQuarterHeaders()]
    
    def reorderQuarters(self):
        years = self.getYears()
        headers = self.getLabelHeaders()
        #years = [year for year in years if "Q1 "+year in self.getQuarterHeaders()]
        quarters = self.getQuarterHeaders()
        cols = [q + " " + year for year in years for q in ["Q1","Q2","Q3","Q4","Year Ended"] if q + " " + year in quarters]
        cols = headers + cols
        self.finalStatement = self.finalStatement[cols]
        
    def getYears(self):
        years = list(set([x.split(" ")[-1] for x in self.getQuarterHeaders()]))
        years.sort()
        return years
    
    
   

In [804]:
class FinancialDataMerge:
    
    def addData(self, FinancialReports):
        for [sheet,file] in FinancialReports:
            print(file)
            dates = [col for col in sheet.columns[1:] if col not in self.finalStatement.columns] ## remove columns of data already in main sheet sheet
            if len(dates) == 0:
                continue
            cols = ["label"]+dates
            sheet = sheet[cols]
            
            sheet = self.parseStatement(sheet)
            
            self.finalStatement = pd.merge(self.finalStatement, sheet, how="outer",on=["label","title"],suffixes=('', '_x'))
            
            self.finalStatement.title_order = [self.merge_number(self.finalStatement.title_order[i],self.finalStatement.title_order_x[i]) for i in range(len(self.finalStatement))] #merge the number for the titles. use greatest num bc bigger num means there is section before it
            self.finalStatement = self.finalStatement.drop(["title_order_x"],axis=1)
    
    def rearrangeFinalStatement(self):
        self.finalStatement = self.finalStatement.replace(" ",np.nan)
        self.finalStatement = self.finalStatement.loc[~self.finalStatement[self.getQuarterHeaders()].isnull().all(axis=1)]
        
        #Make sure each title has same max number index
        self.reconcileTitleOrder()
        
        # put Totals at the bottom of each section
        self.reconcileLabelOrderWithTotals()
        
        self.finalStatement[["title_order","label_order"]] = self.finalStatement[["title_order","label_order"]].astype("int")
        self.finalStatement = self.finalStatement.sort_values(["title_order","label_order"]) # order by title and then location
        
        self.finalStatement = self.finalStatement.drop(["title_order","label_order"],axis=1).reset_index(drop=True)
        
        self.finalStatement[self.getQuarterHeaders()] = self.finalStatement[self.getQuarterHeaders()].fillna(0)
        
    def parseStatement(self,sheet):
        
        sheet = sheet.loc[(~sheet.drop(columns="label").isnull().all(axis=1) | sheet.label.str.contains(":"))]
        num = 0
        index = []
        title = None 
        for label in sheet.label:
            if ":" in label:
                title = re.sub("\([\w\W]+\)","",label[:-1]).replace("  "," ")
                num += 1
                continue
            new_label = label.lower()
            new_label = new_label.replace("gain","loss").replace("decrease","increase")
            new_label = re.sub("\([\w\W]+?\)","",new_label).replace("  "," ").replace("—"," - ").strip()
            new_label = re.sub(" \$|\s*\d+\,*", "", new_label) 
            new_label = self.clean_up_string(new_label)
            
            is_total = self.total_keyword == new_label.split(" ")[0]
            
            index.append([title,num,label,new_label])
                
            if is_total:
                title = None 
                num += 1
                
        sheet = sheet.loc[sheet.label.isin(np.array(index)[:,2])].copy()
        sheet["title"] = np.array(index)[:,0].tolist()
        sheet["title_order"] = np.array(index)[:,1].tolist()
        sheet["label"] = np.array(index)[:,3].tolist()
        sheet = sheet.set_index(["title","title_order","label"]).reset_index()
        self.sheet = sheet
        return sheet
    
        
    def reconcileTitleOrder(self):
        title_groupby = self.finalStatement.groupby("title")["title_order"].max().reset_index().dropna(axis=0, how='any')
        self.finalStatement = self.finalStatement.merge(title_groupby, on="title", how="left", suffixes=('', '_x') )
        self.finalStatement["title_order"] = [self.merge_number(self.finalStatement.title_order[i],self.finalStatement.title_order_x[i]) for i in range(len(self.finalStatement))] #merge the number for the titles. 
        self.finalStatement = self.finalStatement.drop(["title_order_x"],axis=1)
        
    def reconcileLabelOrderWithTotals(self):
        totals = self.finalStatement.loc[self.finalStatement.label.str.startswith(self.total_keyword)].reset_index(drop=True) 
        self.finalStatement = self.finalStatement.loc[~self.finalStatement.label.str.startswith(self.total_keyword)].reset_index(drop=True)
        self.finalStatement = pd.concat([self.finalStatement,totals]) 
        self.finalStatement = self.finalStatement.reset_index(drop=True).reset_index().rename(columns={"index":"label_order"})
        
            

In [800]:
class Income(FinancialDataMerge, ProcessQuarters, HelperFunctions):
    def __init__(self):
        self.total_keyword = "net"
        self.finalStatement = pd.DataFrame(columns=["title","title_order","label"])
        
        self.IncomeStatements = self.getIncomeStatements()
        self.addData(self.IncomeStatements)
        
        self.rearrangeFinalStatement()
        
        self.setQuarters()
        
        self.performIncomeMath()
        
        self.reorderQuarters()
        
      
    def getIncomeStatements(self):
        FinancialReports = self.getFinancialStatements()
        IncomeStatements = [[report.getIncomeSheet().copy(), report.file] for report in FinancialReports]
        return IncomeStatements
    
    def parseQuarter(self,date):
        mo = date.split(" ")[0].strip(".")
        year = date.split(", ")[-1]
        if mo in ["Apr","May"]:
            return "Q1 "+str(int(year)+1)
        
        if mo in ["Jul","Aug"]:
            return "Q2 "+str(int(year)+1)
        
        if mo in ["Oct","Nov"]:
            return "Q3 "+str(int(year)+1)
        
        if mo in ["Jan","Feb"]:
            return "Year Ended "+str(int(year))
        
        print(mo)
        
    
    def performIncomeMath(self):
        quarters = self.finalStatement[self.getQuarterHeaders()]
        self.finalStatement[self.getQuarterHeaders()] = quarters.where(np.abs(quarters)<10**6,quarters/10**6)
        
        years = self.getYears()
        for year in years:
            if "Q1 "+year not in self.getQuarterHeaders(): 
                continue
            self.finalStatement["Q4 "+year] = self.finalStatement["Year Ended "+year] - self.finalStatement["Q3 "+year]
            
            self.finalStatement.loc[self.finalStatement.label.str.startswith("weighted-average"),"Q4 "+year] = self.finalStatement.loc[self.finalStatement.label.str.startswith("weighted-average"),"Year Ended "+year]
            

            

In [801]:
income = Income()
finalIncome = income.finalStatement
finalIncome.to_excel("output/income.xlsx",index=False)
finalIncome



q3 2022 (2021).xlsx
q3 2019 (2018).xlsx
q3 2021 (2020).xlsx
q3 2018 (2017).xlsx
q1 2020 (2019).xlsx
q2 2022 (2021).xlsx
q2 2018 (2017).xlsx
q2 2021 (2020).xlsx
10k 2020 (2019).xlsx
q2 2019 (2018).xlsx
q1 2022 (2021).xlsx
q3 2020 (2019).xlsx
q1 2021 (2020)xlsx.xlsx
q1 2018 (2017).xlsx
q1 2019 (2018).xlsx
10k 2022 (2021).xlsx
10k 2019 (2018).xlsx
q2 2020 (2019).xlsx
10k 2018 (2017).xlsx
10k 2021 (2022).xlsx


,title,label,Year Ended 2016,Q1 2017,Q2 2017,Q3 2017,Q4 2017,Year Ended 2017,Q1 2018,Q2 2018,...,Q1 2021,Q2 2021,Q3 2021,Q4 2021,Year Ended 2021,Q1 2022,Q2 2022,Q3 2022,Q4 2022,Year Ended 2022
0,NaN,revenues,0.00,0.00,0.00,0.00,15516.00,15516.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00
1,NaN,net sales,15797.00,3438.00,3851.00,3798.00,-3798.00,0.00,3440.00,3799.00,...,2107.00,3275.00,3994.00,9806.00,13800.00,3991.00,4211.00,3943.0,12727.00,16670.00
2,NaN,cost of goods sold and occupancy expenses,10077.00,2229.00,2414.00,2305.00,7571.00,9876.00,2137.00,2320.00,...,1839.00,2126.00,2374.00,6721.00,9095.00,2361.00,2388.00,2282.0,7751.00,10033.00
3,NaN,gross profit,5720.00,1209.00,1437.00,1493.00,4147.00,5640.00,1303.00,1479.00,...,268.00,1149.00,1620.00,3085.00,4705.00,1630.00,1823.00,1661.0,4976.00,6637.00
4,NaN,operating expenses,4196.00,987.00,1158.00,1104.00,3345.00,4449.00,1049.00,1028.00,...,1512.00,1076.00,1445.00,4122.00,5567.00,1390.00,1414.00,1508.0,4319.00,5827.00
5,NaN,operating income,1524.00,222.00,279.00,389.00,802.00,1191.00,254.00,451.00,...,-1244.00,73.00,175.00,-1037.00,-862.00,240.00,409.00,153.0,657.00,810.00
6,NaN,loss on extinguishment of debt,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,58.00,0.00,58.00,58.00,0.00,0.00,325.0,0.00,325.00
7,NaN,interest expense,59.00,19.00,18.00,20.00,55.00,75.00,19.00,16.00,...,19.00,58.00,55.00,137.00,192.00,54.00,51.00,44.0,123.00,167.00
8,NaN,interest income,-6.00,-1.00,-2.00,-3.00,-5.00,-8.00,-3.00,-4.00,...,-4.00,-2.00,-1.00,-9.00,-10.00,-1.00,-1.00,-1.0,-4.00,-5.00
9,NaN,income before income taxes,1471.00,204.00,263.00,372.00,752.00,1124.00,238.00,439.00,...,-1259.00,-41.00,121.00,-1223.00,-1102.00,187.00,359.00,-215.0,538.00,323.00


In [754]:
finalIncome.columns

Index(['title', 'label', 'Q1 2019', 'Q2 2019', 'Q3 2019', 'Year Ended 2019',
       'Q1 2020', 'Q2 2020', 'Q3 2020', 'Year Ended 2020', 'Q1 2021',
       'Q2 2021', 'Q3 2021', 'Year Ended 2021', 'Q1 2022', 'Q2 2022',
       'Q3 2022', 'Year Ended 2022', 'Q4 2019', 'Q4 2020', 'Q4 2021',
       'Q4 2022'],
      dtype='object')

In [791]:
class Balance(FinancialDataMerge, ProcessQuarters, HelperFunctions):
    def __init__(self):
        self.total_keyword = "total"
        self.finalStatement = pd.DataFrame(columns=["title","title_order","label"])
        
        self.BalanceStatements = self.getBalanceStatements()
        self.addData(self.BalanceStatements)
        
        self.rearrangeFinalStatement()
        
        self.setQuarters()
        
        self.performBalanceMath()
        
        self.reorderQuarters()
        
      
    def getBalanceStatements(self):
        FinancialReports = self.getFinancialStatements()
        BalanceStatements = [[report.getBalanceSheet().copy(), report.file] for report in FinancialReports]
        return BalanceStatements
    
    def parseQuarter(self,date):
        mo = date.split(" ")[0].strip(".")
        year = date.split(", ")[-1]
        if mo in ["Apr","May"]:
            return "Q1 "+str(int(year)+1)
        
        if mo in ["Jul","Aug"]:
            return "Q2 "+str(int(year)+1)
        
        if mo in ["Oct","Nov"]:
            return "Q3 "+str(int(year)+1)
        
        if mo in ["Jan","Feb"]:
            return "Year Ended "+str(int(year))
        
        
    def performBalanceMath(self):
        years = self.getYears()
        for year in years:
            if "Q1 "+year not in self.getQuarterHeaders(): 
                continue
            self.finalStatement["Q4 "+year] = self.finalStatement["Year Ended "+year]
        

In [792]:
balance = Balance()
finalBalance = balance.finalStatement
finalBalance.to_excel("output/balance.xlsx")
finalBalance


q3 2022 (2021).xlsx
q3 2019 (2018).xlsx
q3 2021 (2020).xlsx
q3 2018 (2017).xlsx
q1 2020 (2019).xlsx
q2 2022 (2021).xlsx
q2 2018 (2017).xlsx
q2 2021 (2020).xlsx
10k 2020 (2019).xlsx
q2 2019 (2018).xlsx
q1 2022 (2021).xlsx
q3 2020 (2019).xlsx
q1 2021 (2020)xlsx.xlsx
q1 2018 (2017).xlsx
q1 2019 (2018).xlsx
10k 2022 (2021).xlsx
10k 2019 (2018).xlsx
q2 2020 (2019).xlsx
10k 2018 (2017).xlsx
10k 2021 (2022).xlsx


,title,label,Q1 2017,Q2 2017,Q3 2017,Q4 2017,Year Ended 2017,Q1 2018,Q2 2018,Q3 2018,...,Q1 2021,Q2 2021,Q3 2021,Q4 2021,Year Ended 2021,Q1 2022,Q2 2022,Q3 2022,Q4 2022,Year Ended 2022
0,Current assets,cash and cash equivalents,1313.0,1681.0,1522.0,1783.0,1783.0,1583.0,1609.0,1353.0,...,1028.0,2188.0,2471.0,1988.0,1988.0,2066.0,2375.0,801.0,877.00,877.00
1,Current assets,short-term investments,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,178.0,410.0,410.0,0.0,0.0,275.0,0.00,0.00
2,Current assets,merchandise inventory,1958.0,1951.0,2398.0,1830.0,1830.0,1961.0,2051.0,2476.0,...,2217.0,2242.0,2747.0,2451.0,2451.0,2370.0,2281.0,2721.0,3018.00,3018.00
3,Current assets,other current assets,674.0,669.0,751.0,702.0,702.0,575.0,598.0,654.0,...,920.0,882.0,966.0,1159.0,1159.0,1091.0,1201.0,1410.0,1270.00,1270.00
4,Current assets,"available-for-sale securities, current",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,51.0,25.0,0.0,0.0,0.0,475.0,337.0,0.0,0.00,0.00
5,Current assets,total current assets,3945.0,4301.0,4671.0,4315.0,4315.0,4119.0,4258.0,4483.0,...,4216.0,5337.0,6362.0,6008.0,6008.0,6002.0,6194.0,5207.0,5165.00,5165.00
6,nan,"property and equipment, net of accumulated dep...",2864.0,2755.0,2662.0,2616.0,2616.0,2605.0,2643.0,2686.0,...,2945.0,2895.0,2846.0,2841.0,2841.0,2839.0,2897.0,2924.0,3037.00,3037.00
7,nan,"operating lease, right-of-use asset",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4851.0,4689.0,4460.0,4217.0,4217.0,4060.0,3975.0,3788.0,0.00,0.00
8,nan,other long-term assets,698.0,681.0,674.0,679.0,679.0,687.0,716.0,726.0,...,698.0,795.0,705.0,703.0,703.0,703.0,693.0,861.0,884.00,884.00
9,nan,"accumulated depreciation, depletion and amorti...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5071.00,5071.00


In [802]:
class CashFlow(FinancialDataMerge, ProcessQuarters, HelperFunctions):
    def __init__(self):
        self.total_keyword = "net"
        self.finalStatement = pd.DataFrame(columns=["title","title_order","label"])
        
        self.CashFlowStatements = self.getCashFlowStatements()
        self.addData(self.CashFlowStatements)
        
        self.rearrangeFinalStatement()
        
        self.setQuarters()
        
        self.performCashFlowMath()
        self.reorderQuarters()
        
      
    def getCashFlowStatements(self):
        FinancialReports = self.getFinancialStatements()
        CashFlowStatements = [[report.getCashFlowSheet().copy(), report.file] for report in FinancialReports]
        return CashFlowStatements
    
    def parseQuarter(self,date):
        mo = date.split(" ")[0].strip(".")
        year = date.split(", ")[-1]
        if mo in ["Apr","May"]:
            return "Q1 "+str(int(year)+1)
        
        if mo in ["Jul","Aug"]:
            return "6mo "+str(int(year)+1)
        
        if mo in ["Oct","Nov"]:
            return "9mo "+str(int(year)+1)
        
        if mo in ["Jan","Feb"]:
            return "Year Ended "+str(int(year))     
    def performCashFlowMath(self):
        years = self.getYears()
        for year in years:
            if "Q1 "+year not in self.getQuarterHeaders(): 
                continue
            # find Q2 
            self.finalStatement["Q2 "+year] = self.finalStatement["6mo "+year] - self.finalStatement["Q1 "+year]
            
            #find Q3 
            self.finalStatement["Q3 "+year] = self.finalStatement["9mo "+year] - self.finalStatement["6mo "+year]
            
            #find Q3 
            self.finalStatement["Q4 "+year] = self.finalStatement["Year Ended "+year] - self.finalStatement["9mo "+year]
    
            
        

In [803]:
cashflow = CashFlow()
finalCashflow = cashflow.finalStatement
finalCashflow.to_excel("output/cashflow.xlsx")
finalCashflow

q3 2022 (2021).xlsx
q3 2019 (2018).xlsx
q3 2021 (2020).xlsx
q3 2018 (2017).xlsx
q1 2020 (2019).xlsx
q2 2022 (2021).xlsx
q2 2018 (2017).xlsx
q2 2021 (2020).xlsx
10k 2020 (2019).xlsx
q2 2019 (2018).xlsx
q1 2022 (2021).xlsx
q3 2020 (2019).xlsx
q1 2021 (2020)xlsx.xlsx
q1 2018 (2017).xlsx
q1 2019 (2018).xlsx
10k 2022 (2021).xlsx
10k 2019 (2018).xlsx
q2 2020 (2019).xlsx
10k 2018 (2017).xlsx
10k 2021 (2022).xlsx


,title,label,Year Ended 2016,Q1 2017,Q2 2017,Q3 2017,Q4 2017,Year Ended 2017,Q1 2018,Q2 2018,...,Q1 2021,Q2 2021,Q3 2021,Q4 2021,Year Ended 2021,Q1 2022,Q2 2022,Q3 2022,Q4 2022,Year Ended 2022
0,Cash flows from operating activities,net income,920,127,125,204,220,676,143,271,...,-932,-62,95,234,-665,166,258,-152,-16,256
1,Adjustments to reconcile net income to net cas...,depreciation and amortization,592,148,155,146,144,593,138,141,...,130,126,125,126,507,120,124,128,132,504
2,Adjustments to reconcile net income to net cas...,share-based compensation,76,15,21,19,21,76,20,22,...,18,17,20,22,77,36,36,25,42,139
3,Adjustments to reconcile net income to net cas...,"operating lease, impairment loss",0,0,0,0,0,0,0,0,...,360,1,0,-361,0,5,1,0,-6,0
4,Adjustments to reconcile net income to net cas...,other asset impairment charges,54,0,0,0,107,107,0,0,...,124,3,0,-127,0,0,1,0,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,Supplemental disclosure of cash flow information,"interest paid, excluding capitalized interest,...",0,0,0,0,0,0,0,0,...,38,1,2,-41,0,2,100,76,-178,0
95,Supplemental disclosure of cash flow information,"cash paid for income taxes during the period, ...",452,43,100,175,170,488,35,95,...,37,16,-45,12,20,20,127,34,34,215
96,Supplemental disclosure of cash flow information,cash paid for interest during the period,78,39,2,39,2,82,38,0,...,0,0,0,145,145,0,0,0,180,180
97,Supplemental disclosure of cash flow information,"operating lease, payments",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
